In [1]:
import requests
import json
import time
import math
import re
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import os, sys, glob
import kaleido
from PIL import Image
from fpdf import FPDF
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [3]:
def viz(data):
    fig = go.Figure(data=[go.Candlestick(x=data['UTC_Time'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False,
                      title_font_color="blue",
                      title_font_size = 20)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])   
    fig.show()

In [4]:
def find_k_similar_candles(candle_id, dataset, k=4):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'auto') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i!=0:
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
#                           dataset.iloc[indices.flatten()[i]]['O-H'],dataset.iloc[indices.flatten()[i]]['O-L'],dataset.iloc[indices.flatten()[i]]['O-C'],dataset.iloc[indices.flatten()[i]]['H-L'],dataset.iloc[indices.flatten()[i]]['H-C'],dataset.iloc[indices.flatten()[i]]['L-C'],
                           dataset.iloc[indices.flatten()[i]]['col_1'],dataset.iloc[indices.flatten()[i]]['col_2'],dataset.iloc[indices.flatten()[i]]['col_3'],
#                           dataset.iloc[indices.flatten()[i]]['col_4'],dataset.iloc[indices.flatten()[i]]['col_5'],
                           dataset.iloc[indices.flatten()[i]]['F_SMA_10'],dataset.iloc[indices.flatten()[i]]['F_SMA_20'],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
#                      'O-H','O-L','O-C','H-L','H-C','L-C',
                      'col_1','col_2','col_3',
#                      'col_4','col_5',
                      'F_SMA_10','F_SMA_20'
                     ]
   # display (output)
    
    return indices, distances

# <font color='red'>Test Configs</font>

In [5]:
Test_Candle = [2108, 8371, 4614, 8207, 5172, 3342, 121, 8607, 6387, 9303, 3835, 2134, 6930, 6858, 7301, 8099, 985, 5419, 7471, 1457, 8910, 3841, 704, 7308, 8734, 7172, 7385, 1613, 5421, 7061, 1088, 4383, 9123, 166, 9414, 3899, 4915, 988, 5856, 4308, 742, 5213, 6201, 3898, 1346, 9255, 2498, 6305, 4755, 837, 4628, 805, 3896, 5168, 237, 6147, 4220, 3683, 6668, 989, 216, 4455, 3231, 3456, 8141, 629, 8250, 8351, 8648, 2499, 1818, 1253, 22, 3337, 8189, 2423, 5209, 1426, 4332, 3663, 8871, 1620, 8493, 7271, 214, 6443, 97, 2524, 1767, 7839, 3644, 6230, 7605, 4921, 5945, 2162, 3773, 7216, 2936]



In [6]:
filename = 'EUR_USD_H4.csv'
data = pd.read_csv(filename)

In [7]:
data.shape

(9653, 29)

In [8]:
data.head(2)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,Direction,col_1,col_2,col_3,col_4,col_5,SMA_10,SMA_20,F_SMA_10,F_SMA_20
0,2015-08-19,01:00:00,1,231,2,Wednesday,2015-08-19T01:00:00.000000000Z,1665,1.10380,1.10526,...,1,-0.00109,0.00037,0.00019,-0.00138,0.00052,1.103502,1.105358,0.001388,-0.000468
1,2015-08-19,05:00:00,5,231,2,Wednesday,2015-08-19T05:00:00.000000000Z,4532,1.10492,1.10734,...,1,-0.00046,0.00196,0.00009,-0.00109,-0.00138,1.103942,1.105107,0.001438,0.000273


# <font color='red'>CANDLE LOOP</font>

In [9]:
Predicted_Market = [] 
result_output = pd.DataFrame({'Candle_No':[],
                              'Current_Market_Fit':[],
                              'Current_Market':[],
                              'Current_Market_Stoploss':[]
                             })

for candle_no in Test_Candle:
    current_market = "currnet_market.csv"
    data = pd.read_csv(filename)
    data = data.iloc[candle_no:candle_no+10]
    data.to_csv(current_market, header = True, index = False)
    data = pd.read_csv(current_market)

    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = r2_score(Y, y_pred).round(2)*100 # 1.  Saved Data -> current market , fit
    #print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market='Bullish'    #  2. Saved Data  -> current market
        Current_Market_Stoploss= 0 #BUY_SL #  3. Saved Data  -> current market , buy stoploss
    else:
        Current_Market = 'Bearish'     # 2. Saved Data  -> current market
        Current_Market_Stoploss= 0 #SELL_SL # 3. Saved Data  -> current market , sell stoploss
    
    result = {'Candle_No': candle_no,
              'Current_Market_Fit': Current_Market_Fit,
              'Current_Market': Current_Market,
              'Current_Market_Stoploss':Current_Market_Stoploss}
    
    result_output = result_output.append(result, ignore_index = True)

In [10]:
result_output.to_csv('Back_Test_Result.csv', header = True, index = False)
result_output = pd.read_csv('Back_Test_Result.csv')